In [1]:
import astropy ; print(astropy.__version__)
import photutils ; print(photutils.__version__)

import matplotlib.pyplot as plt

import tng_api_utils as tau

import os
import numpy as np

3.2.1
0.7.2


In [2]:
# Probably need more stuff here

Measure stuff from catalogs

In [ ]:
massive_primaries_im = mstar_tbl['source_sum'] >= (10.0**9.5)
possible_secondaries_im = mstar_tbl['source_sum'] >= (10.0**7.5)
print(np.sum(massive_primaries_im),np.sum(possible_secondaries_im))

In [ ]:
massive_primaries_cat = sam_catalog['mstar'] >= (10.0**9.5)/(10.0**10.0)
possible_secondaries_cat = sam_catalog['mstar'] >= (10.0**7.5)/(10.0**10.0)
print(np.sum(massive_primaries_cat),np.sum(possible_secondaries_cat))

In [ ]:
zi = sam_z[indices] >= 2.0
print(sam_z[indices].shape,sam_z.shape,mstar_tbl['source_sum'].shape)
mzi=np.logical_and(massive_primaries_im==True,zi==True)
print(np.sum(mzi))

In [ ]:
print(sam_catalog.colnames)
print(mstar_tbl.colnames)
print(input_skycoords.shape,sam_z.shape)

In [ ]:
def count_pairs(pri_z,pri_m,pri_sc,sec_z,sec_m,sec_sc,pri_kpcas,d_kpc=50.0,zfrac=0.02,mfrac=0.1):
    #print(pri_z,pri_m)
    sep = pri_sc.separation(sec_sc)
    sep_kpc = pri_kpcas*sep.to('arcsec').value
    slist = np.logical_and(sep_kpc > 1.0, np.logical_and(np.logical_and(sep_kpc < d_kpc,np.abs(sec_z-pri_z)<zfrac*(1.0+pri_z)),sec_m/pri_m >= mfrac))
    num_pairs=np.sum(slist)
    return num_pairs, sep_kpc

In [ ]:
zbins=((1.0,2.0),(2.0,2.5),(2.5,3.0),(3.0,4.0),(4.0,5.0),(5.0,6.0),(6.0,8.0))
zpts =(1.5,2.25,2.75,3.5,4.5,5.5,7.0)
fp_im=[]
fp_cat=[]
for zb in zbins:
    zl=zb[0] ; zr=zb[1]
    this_zi= np.logical_and(sam_z[indices]>=zl,sam_z[indices]<zr)
    this_zi_c = np.logical_and(sam_z>=zl,sam_z<zr)
    
    this_pzi=np.logical_and(massive_primaries_im==True,this_zi==True)
    this_szi=np.logical_and(possible_secondaries_im==True,this_zi==True)
    
    this_pzi_c=np.logical_and(massive_primaries_cat==True,this_zi_c==True)
    this_szi_c=np.logical_and(possible_secondaries_cat==True,this_zi_c==True)
    
    pri_indices = np.where(this_pzi==True)[0]
    sec_indices = np.where(this_szi==True)[0]
    
    pri_indices_c = np.where(this_pzi_c==True)[0]
    sec_indices_c = np.where(this_szi_c==True)[0]
        
    pairs_this_z=0
    pri_this_z=0
    skl=[]
    
    for pi in pri_indices:
        pri_m=mstar_tbl['source_sum'][pi]
        pri_z=sam_z[indices][pi]
        pri_sc=source_skycoords[pi]
        pri_kpcas=sam_catalog['kpc_per_arcsec'][indices][pi]
        sec_m=mstar_tbl['source_sum'][sec_indices]
        sec_z=sam_z[indices][sec_indices]
        sec_sc=source_skycoords[sec_indices]
        npairs,sk = count_pairs(pri_z,pri_m,pri_sc,sec_z,sec_m,sec_sc,pri_kpcas)
        pri_this_z+=1
        pairs_this_z+=npairs
        skl.append(sk)
    #print(np.median(np.asarray(skl)),np.min(np.asarray(skl)))
        
    pairs_this_z_c=0
    pri_this_z_c=0
    for pi in pri_indices_c:
        pri_m=sam_catalog['mstar'][pi]
        pri_z=sam_z[pi]
        pri_sc=input_skycoords[pi]
        pri_kpcas=sam_catalog['kpc_per_arcsec'][pi]
        sec_m=sam_catalog['mstar'][sec_indices_c]
        sec_z=sam_z[sec_indices_c]
        sec_sc=input_skycoords[sec_indices_c]
        npairs,sk = count_pairs(pri_z,pri_m,pri_sc,sec_z,sec_m,sec_sc,pri_kpcas)
        pri_this_z_c+=1
        pairs_this_z_c+=npairs
        skl.append(sk)
    #print(np.median(np.asarray(skl)),np.min(np.asarray(skl)))        

    if len(pri_indices) > 0:
        print(zb,pri_this_z,pairs_this_z,pairs_this_z/pri_this_z, pairs_this_z_c/pri_this_z_c)
        fp_im.append(pairs_this_z/pri_this_z)
        fp_cat.append(pairs_this_z_c/pri_this_z_c)
    else:
        fp_im.append(0.0)
        fp_cat.append(0.0)

In [ ]:
fig, ((ax1)) = plt.subplots(1, 1, figsize=(12, 8),dpi=300)
ax1.plot(zpts,fp_im,marker='o',linestyle='None',markersize=20,markerfacecolor='Black',mec='None')
ax1.plot(zpts,fp_cat,marker='s',linestyle='None',markersize=20,markerfacecolor='Orange',mec='None')
ax1.legend(('Image-Based', 'Catalog-Based'),loc='upper left',fontsize=20)
ax1.set_xlabel('redshift',size=25)
ax1.set_ylabel('Pair Fraction',size=25)
ax1.set_ylim(0.0,1.60)
ax1.tick_params(labelsize=25)
fig.savefig('F356W/SAM_Pairs_Im_vs_Cat_Mass95.png',dpi=300)

In [ ]:
ubins=((20.0,15.0),(15.0,10.0),(10.0,7.0),(7.0,4.0),(4.0,2.0),(2.0,1.0))
upts =(20.0,15.0,10.0,7.0,4.0,2.0)
fp_im=[]
fp_cat=[]
for ub in ubins:
    ul=ub[0] ; ur=ub[1]
    zl=3.0
    zr=10.0
    this_zi= np.logical_and(sam_z[indices]>=zl,sam_z[indices]<zr)
    this_zi_c = np.logical_and(sam_z>=zl,sam_z<zr)
    
    this_pzi=np.logical_and(massive_primaries_im==True,this_zi==True)
    this_szi=np.logical_and(possible_secondaries_im==True,this_zi==True)
    
    this_pzi_c=np.logical_and(massive_primaries_cat==True,this_zi_c==True)
    this_szi_c=np.logical_and(possible_secondaries_cat==True,this_zi_c==True)
    
    pri_indices = np.where(this_pzi==True)[0]
    sec_indices = np.where(this_szi==True)[0]
    
    pri_indices_c = np.where(this_pzi_c==True)[0]
    sec_indices_c = np.where(this_szi_c==True)[0]
        
    pairs_this_z=0
    pri_this_z=0
    skl=[]
    
    for pi in pri_indices:
        pri_m=mstar_tbl['source_sum'][pi]
        pri_z=sam_z[indices][pi]
        pri_sc=source_skycoords[pi]
        pri_kpcas=sam_catalog['kpc_per_arcsec'][indices][pi]
        sec_m=mstar_tbl['source_sum'][sec_indices]
        sec_z=sam_z[indices][sec_indices]
        sec_sc=source_skycoords[sec_indices]
        npairs,sk = count_pairs(pri_z,pri_m,pri_sc,sec_z,sec_m,sec_sc,pri_kpcas,mfrac=1.0/ul)
        pri_this_z+=1
        pairs_this_z+=npairs
        skl.append(sk)
    #print(np.median(np.asarray(skl)),np.min(np.asarray(skl)))
        
    pairs_this_z_c=0
    pri_this_z_c=0
    for pi in pri_indices_c:
        pri_m=sam_catalog['mstar'][pi]
        pri_z=sam_z[pi]
        pri_sc=input_skycoords[pi]
        pri_kpcas=sam_catalog['kpc_per_arcsec'][pi]
        sec_m=sam_catalog['mstar'][sec_indices_c]
        sec_z=sam_z[sec_indices_c]
        sec_sc=input_skycoords[sec_indices_c]
        npairs,sk = count_pairs(pri_z,pri_m,pri_sc,sec_z,sec_m,sec_sc,pri_kpcas,mfrac=1.0/ul)
        pri_this_z_c+=1
        pairs_this_z_c+=npairs
        skl.append(sk)
    #print(np.median(np.asarray(skl)),np.min(np.asarray(skl)))        

    if len(pri_indices) > 0:
        print(zb,pri_this_z,pairs_this_z,pairs_this_z/pri_this_z, pairs_this_z_c/pri_this_z_c)
        fp_im.append(pairs_this_z/pri_this_z)
        fp_cat.append(pairs_this_z_c/pri_this_z_c)
    else:
        fp_im.append(0.0)
        fp_cat.append(0.0)

In [ ]:
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

fig, ((ax1)) = plt.subplots(1, 1, figsize=(12, 8),dpi=300)
ax1.loglog(upts,fp_im,marker='o',linestyle='None',markersize=20,markerfacecolor='Black',mec='None')
ax1.loglog(upts,fp_cat,marker='s',linestyle='None',markersize=20,markerfacecolor='Orange',mec='None')
ax1.legend(('Image-Based', 'Catalog-Based'),loc='upper left',fontsize=20)
ax1.set_xlabel(r'$M_1/M_2$',size=25)
ax1.set_ylabel('Pair Fraction',size=25)
ax1.set_ylim(0.05,2.0)
ax1.set_xlim(25.0,1.7)
ax1.set_xticks(upts)
ax1.set_xticklabels([20,15,10,7,4,2])
ax1.yaxis.set_major_formatter(FormatStrFormatter('%4.1f'))
ax1.tick_params(labelsize=25)
ax1.tick_params(which='minor',bottom=False)
fig.savefig('F356W/SAM_Ratios_Im_vs_Cat_Mass95_z3_29.png',dpi=300)